# A LOOK AT KERAS DATASETS

In [ ]:
# keras has built-in datasets
import keras.datasets.mnist

In [ ]:
#### MNIST

In [ ]:
# let's load mnist dataset
data_mnist = keras.datasets.mnist.load_data()

In [ ]:
# let's check the shape of this nd second : Vector of size : 60000
print(data_mnist[1][1].shape)

In [ ]:
data_mnist[0][1].shape

In [ ]:
# the type of the dataset is a tuple (we can guess that there is training and test dataset) 
print(type(data_mnist))

In [ ]:
# the length of the dataset tuple is 2 even more sure that there is training and test dataset) 
print(len(data_mnist))

In [ ]:
# let's access the first element an check its type
print(type(data_mnist[0]))

In [ ]:
# this is a 2 tuple -> chances  are that it contains data and the label
print(len(data_mnist[0]))

In [ ]:
#### X_train : first element of the first element in the parent 2-tuple:

In [ ]:
# let's check the size of the first element : we red 60 k observations
print(len(data_mnist[0][0]))

In [ ]:
# let's check the type of the first element : it is a n-dimensional array
print(type(data_mnist[0][0]))

In [ ]:
# let's check the shape of this nd array : tensor of order 3 : 60000x28x28
print(data_mnist[0][0].shape)

In [ ]:
#### Y_train : second element of the first elment in the parent 2-tuple:

In [ ]:
# let's check the size of the second element : we red 60 k observations
print(len(data_mnist[0][1]))

In [ ]:
# let's check the type of the first second : it is a n-dimensional array
print(type(data_mnist[0][1]))

In [ ]:
# let's check the shape of this nd second : Vector of size : 60000
print(data_mnist[0][1].shape)

In [ ]:
#### X_test : first element of the second element in the parent 2-tuple:

In [ ]:
# let's check the size of the first element : we red 10 k observations
print(len(data_mnist[1][0]))

In [ ]:
# let's check the type of the first element : it is a n-dimensional array
print(type(data_mnist[1][0]))

In [ ]:
# let's check the shape of this nd second : tensor of order 3 : 10000x28x28
print(data_mnist[1][0].shape)

In [ ]:
#### Y_test : second element of the second element in the parent 2-tuple:

In [ ]:
# let's check the size of the first element : we red 10 k observations
print(len(data_mnist[1][1]))

In [ ]:
# let's check the type of the first element : it is a n-dimensional array
print(type(data_mnist[1][1]))

In [ ]:
## BUILD CLASSES (DATASETS)

In [ ]:
class myDataset():
    def __init__(self,data):
        # the class propperty X_train is populated  with the elements of data
        # as explained above
        self.X_train = data[0][0]
        self.Y_train = data[0][1]
        self.X_test = data[1][0]
        self.Y_test = data[1][1]
    
    def get_stats(self):
        # this function displays the number of observations
        my_str = "there are {0} training observations, {1} test observations".format(len(self.X_train), len(self.X_test))
        print(my_str)
        
    def check_dataset_health(self):
        # checkup function which checks that X and Y have the same number of observations
        assert self.X_train.shape[0] == self.Y_train.shape[0]
        assert self.X_test.shape[0] == self.Y_test.shape[0]
        print("dataset is ok")
        
    def preview(self, image_index =0):
        # display the observation @ a chosen index
        label_at_index = self.Y_train[image_index]
        plt.title("image index n° {0} and the associated label is {1}".format(image_index, label_at_index))
        plt.imshow(self.X_train[image_index], cmap='gray')
        plt.show()

In [ ]:
# instantiate new Dataset class
new_mnist_dataset = myDataset(data_mnist)

In [ ]:
new_mnist_dataset.X_train.shape

In [ ]:
new_mnist_dataset.Y_train.shape

In [ ]:
# check the new instance health
new_mnist_dataset.check_dataset_health()

In [ ]:
# get the stats of the new dataset 
new_mnist_dataset.get_stats()

In [ ]:
# get the visual of an image at index =k 
new_mnist_dataset.preview(image_index =10)

In [ ]:
EXERCICE 1:
- Try to recreate the same apporach for FashionMNIST dataset
-  Try to add additional info in the stats : image size,  class distribution (how many observations par class)

In [ ]:
## MODELS

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import optimizers
from keras.models import Model
from keras.layers import Flatten,Dense,Dropout
import numpy as np
import keras

class My_predictive_model():
    
    def mod_definition(self):
        # define inputs size
        img_width, img_height = 48, 48
        
        # model without output (transfert learning)
        model = VGG16(weights = None, include_top=False, input_shape = (img_width, img_height, 1))

        # freeze all layers
        for layer in model.layers:
            layer.trainable = False

        # Adding custom Layers 
        x = model.output
        x = Flatten()(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        predictions = Dense(10, activation="softmax")(x)
        
        #final model using the VGG16 structure to which we added a block
        model_final = Model(inputs = model.input, outputs = predictions)

        # compile the model ( define the loss, the optimizer and the monitored metrics)
        adadelta = optimizers.Adadelta(lr=0.001)
        model_final.compile(loss = "categorical_crossentropy", optimizer = adadelta, metrics=["accuracy"])

        return(model_final)
    

    def __init__(self):
        #when the model is initialized it calls the function (mod_definition)
        self.model = self.mod_definition()
        
        
    def train_model(self,x_train,y_train):
        # when the model is trained : in this case it fits x_train, y_train - in memory  (not done in practice)
        # specify 20 epochs and the batch size here : 10
        # this could be changed or parameterized
        
        self.model.fit(x_train, y_train,
          epochs=20,
          batch_size=10)
        
        
    def predict_img(self, img):
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        return self.model.predict(x)
    
    def predict_path(self, path):
        img = image.load_img(img_path, target_size=(48, 48))
        return self.model.predict_img(img)

In [ ]:
# instantiate the model
pred_model = My_predictive_model()

In [ ]:
list_output =[]
# we will resize the images to the target size
for el in tqdm.tqdm(new_mnist_dataset.X_train):
    list_output.append(cv2.resize(el, (48,48)))

# we will output the "stacked images as an array" 
list_output = np.array(list_output)

In [ ]:
# preparation des labels  to be converted in 1-hot encoding
categorical = keras.utils.to_categorical(new_mnist_dataset.Y_train,num_classes=10)
# preparation des images en input to have 4 dimensions ( requirement for conv2D networks)
expanded = np.expand_dims(list_output, axis=-1)

In [ ]:
#### TRAINING (prediction on 1 image)

In [ ]:
# train model
pred_model.train_model(expanded, categorical)

In [ ]:
#### TESTING (prediction on 1 image)

In [ ]:
#let's use a given image
img_path = '/Users/assansanogo/Downloads/elephant.jpeg'
#let's load the image
img = image.load_img(img_path, target_size=(48, 48))

In [ ]:
# there are 10 predictions (the sum = 1)
print(pred_model.predict_img(img))

In [ ]:
# the  final answer (to : "what number is it like ?"")
label_pred = np.argmax(pred_model.predict_img(img))

In [ ]:
# class variation

In [ ]:
class myDataset():
    def __init__(dataset_name, file_path_link):
        self.ds_name = dataset_name
        self.link = file_path_link
        self.path_unzip = os.getcwd()
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.Y_test = None
        self.csv_file_path = ""
        
    def download():
        # here is an url of your choice 
        url = self.link
        # here is a destination path
        file_zip_path = os.path.join(self.path_unzip,self.ds_name)
        os.makedirs(file_zip_path, exist_ok = True)

        # we want to check that the url is valid before downloading
        with urlrequest.urlopen(url) as response:
            resp_status = response.status
        
            # if the error code is 200 we process to downloading else we don't
            if resp_status ==200 and not os.path.exists(file_zip_path):
                # download the file if the response code is 200
                file_,resp = urlrequest.urlretrieve(url)

            elif os.path.exists(file_jpg_path):
                print("Please check the destination folder, the image already exist")
            else:
                print("Please check the link, the download link is not downloadable or does not exist")
    
    def unzip_file():
        with zipfile.ZipFile(file_zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.path_unzip)      

In [1]:
# Build your own !!